# BULKI_Store Demo

## Starting Server

In [1]:
print("Let's start")

Let's start


In [2]:
# import subprocess

# # Start the server as a background process
# server_process = subprocess.Popen(
#     ['/opt/homebrew/bin/mpirun', '-np', '4', 'target/debug/bulkistore-server'],
#     env={'RUST_LOG': 'debug'},
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE
# )

# print("Server started with PID:", server_process.pid)


## Creating Arrays

In [3]:
import bkstore_client as bkc
import numpy as np
import time
from pprint import pprint


bkc.init()
dim_size = 15

print("5D array size = {} elements".format(dim_size ** 5))

# dim_size = 10
rd1 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)
rd2 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)

print("rd1.shape=", rd1.shape)
print("rd1 last 2 dim preview:")
print(rd1[0:1, 0:1, 0:1, 0:2, 0:])

print("rd2.shape=", rd2.shape)
print("rd2 last 2 dim preview:")
print(rd2[0:1, 0:1, 0:1, 0:2, 0:])

arr3 = np.random.rand(dim_size, dim_size, dim_size)
print("arr3.shape=", arr3.shape)
print("arr3 last 2 dim preview:")
print(arr3[0:1, 0:2, 0:])

5D array size = 759375 elements
rd1.shape= (15, 15, 15, 15, 15)
rd1 last 2 dim preview:
[[[[[0.63487986 0.39722519 0.89204678 0.69139252 0.11316525 0.15991436
     0.30718189 0.98112874 0.67969073 0.71671272 0.95756795 0.72825365
     0.96229751 0.29918428 0.51969284]
    [0.6541042  0.98333975 0.49881026 0.74432668 0.54207219 0.3797532
     0.84271896 0.04506763 0.10509971 0.82011961 0.35145049 0.64535071
     0.36304225 0.14489243 0.74171221]]]]]
rd2.shape= (15, 15, 15, 15, 15)
rd2 last 2 dim preview:
[[[[[0.50194098 0.39146545 0.91225314 0.90773776 0.06018236 0.16375427
     0.01620787 0.46945395 0.713789   0.13769742 0.94161528 0.43036764
     0.76832675 0.56643982 0.00130541]
    [0.62273594 0.57367513 0.41281624 0.59901267 0.46244618 0.88587863
     0.31697565 0.91402963 0.82247175 0.24428245 0.99752891 0.46129467
     0.34035222 0.28964249 0.13728861]]]]]
arr3.shape= (15, 15, 15)
arr3 last 2 dim preview:
[[[0.01492931 0.04777806 0.47823219 0.45233998 0.63857488 0.47877315
   0.9

## Client-Side NDArray Arithmetic Operation

In [4]:
arr4 = bkc.polymorphic_add(rd1, rd2)

print("arr4.shape=", arr4.shape)
print("arr4 last 2 dim preview:")
print(arr4[0:1, 0:1, 0:1, 0:2, 0:])



arr4.shape= (15, 15, 15, 15, 15)
arr4 last 2 dim preview:
[[[[[1.13682084 0.78869064 1.80429991 1.59913029 0.17334761 0.32366863
     0.32338976 1.45058269 1.39347973 0.85441015 1.89918323 1.15862129
     1.73062426 0.86562409 0.52099825]
    [1.27684014 1.55701487 0.91162651 1.34333935 1.00451837 1.26563182
     1.15969461 0.95909726 0.92757146 1.06440206 1.34897939 1.10664538
     0.70339447 0.43453491 0.87900082]]]]]


## Remote Array Slicing

In [5]:
arr5 = bkc.array_slicing(arr4, [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 10, -2)])

print("arr5.shape=", arr5.shape)
print("arr5 last 2 dim preview:")
print(arr5[0:1, 0:1, 0:1, 0:2, 0:])



arr5.shape= (1, 1, 1, 2, 5)
arr5 last 2 dim preview:
[[[[[0.85441015 1.45058269 0.32366863 1.59913029 0.78869064]
    [1.06440206 0.95909726 1.26563182 1.34333935 1.55701487]]]]]


## Remote Array Arithmetic Operation

In [6]:
arr6 = bkc.times_two(arr5)

print("arr6.shape=", arr6.shape)
print("arr6:")
print(arr6)

arr6.shape= (1, 1, 1, 2, 5)
arr6:
[[[[[1.7088203  2.90116538 0.64733726 3.19826057 1.57738129]
    [2.12880412 1.91819452 2.53126364 2.68667869 3.11402974]]]]]


## Creating Object with Multiple NDArrays

1. You can attach a series of metadata attributes directly to the object you are creating
2. You can attach a series of NDArrays directly to the object you are creating along with the metadata attributes of these NDArrays.
3. Each of these NDArrays will become a sub-object of the main object you are creating.

In [7]:
# Create an 3D array

name_id=str(np.random.rand(1)[0]*1000000)[0:6]

gnn_arr1=np.random.rand(100000,2)
gnn_arr2=np.random.rand(1,100000)

obj_ids = bkc.create_objects(
    obj_name_key="name",
    parent_id=None,
    metadata={
        "name": "{}".format(name_id),
        "type": "container",
        "keys": [
            "{}/arr6".format(name_id), 
            "{}/arr5".format(name_id), 
            "{}/arr4".format(name_id),
            "{}/arr3".format(name_id),
            "{}/gnn_arr1".format(name_id),
            "{}/gnn_arr2".format(name_id)
                ],
        "ranges": [(1, 100), (100, 200)],
        "part_num": 1,
        "part_size": 100,
    },
    data=arr3,
    array_meta_list=[
        {
            "name": "{}/arr6".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr5".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr4".format(name_id),
            "type": "array",
            "shape": arr4.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr3".format(name_id),
            "type": "array",
            "shape": arr3.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr1".format(name_id),
            "type": "array",
            "shape": gnn_arr1.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr2".format(name_id),
            "type": "array",
            "shape": gnn_arr2.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        }
    ],
    array_data_list=[arr6, arr5, arr4, arr3, gnn_arr1, gnn_arr2],
)
print(obj_ids)

[32110640314422103883843739242491034, 32110640314422712626398175952661658, 32110640314422970880815212181351578, 32110640314423173795000027281386650, 32110640314433208823776129572433050, 32110640314433356397728723543813274, 32110640314436086515851636852419738]


## Retrieve Metadata

### Retrieve a single metadata attributes from a single object


In HydraGNN, we need to retrieve a series of metadata attributes from a container. 

If every single metadata retrieval requires a single request, that would be too much overhead.

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.


In [8]:
meta_keys=["type", "keys", "ranges", "name", "part_num", "part_size"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=[meta_keys[k]], # retreiving SINGLE!!!
        
    )
    if i < len(meta_keys):
        print(result)

print("Single Metadata retrieval of single object : TPS = {} MetaPS: {}".format(1000/(time.time()-start), 1000/(time.time()-start)))

{'obj_id': 32110640314422103883843739242491034, 'metadata': {'type': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422103883843739242491034, 'metadata': {'keys': ['689609/arr6', '689609/arr5', '689609/arr4', '689609/arr3', '689609/gnn_arr1', '689609/gnn_arr2']}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422103883843739242491034, 'metadata': {'ranges': [(1, 100), (100, 200)]}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422103883843739242491034, 'metadata': {'name': '689609'}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422103883843739242491034, 'metadata': {'part_num': 1}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422103883843739242491034, 'metadata': {'part_size': 100}, 'sub_obj_metadata': None}
Single Metadata retrieval of single object : TPS = 1447.6995188491037 MetaPS: 1447.6975201089042


### Retrieving multiple metadata attributes from a single object

Now we can combine the metadata retrieval of a single container into one single request. 

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.

Slight drop on TPS but MEGA IMPROVEMENT for MetaPS

In [9]:
meta_keys=["type", "keys", "ranges", "name", "part_num", "part_size"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=meta_keys, # retrieving MULTIPLE!!!
    )
pprint(result)
print("\n Multiple Metadata retrieval of single object : TPS={}, MetaPS: {} ".format(1000/(time.time()-start), 1000*len(meta_keys)/(time.time()-start) ))

{'metadata': {'keys': ['689609/arr6',
                       '689609/arr5',
                       '689609/arr4',
                       '689609/arr3',
                       '689609/gnn_arr1',
                       '689609/gnn_arr2'],
              'name': '689609',
              'part_num': 1,
              'part_size': 100,
              'ranges': [(1, 100), (100, 200)],
              'type': 'container'},
 'obj_id': 32110640314422103883843739242491034,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of single object : TPS=1292.1732754557245, MetaPS: 7753.030098578714 


### Retrieving single attributes from a single sub-object

In [10]:
meta_keys=["name",
            "type",
            "shape",
            "vcount",
            "voffset",
            "vdim"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_ids[1],
        meta_keys=[meta_keys[k]], # retreiving SINGLE!!!
        
    )
    if i < len(meta_keys):
        print(result)

print("Single Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(1000/(time.time()-start), 1000/(time.time()-start)))

{'obj_id': 32110640314422712626398175952661658, 'metadata': {'name': '689609/arr6'}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422712626398175952661658, 'metadata': {'type': 'array'}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422712626398175952661658, 'metadata': {'shape': [1, 1, 1, 2, 5]}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422712626398175952661658, 'metadata': {'vcount': 100}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422712626398175952661658, 'metadata': {'voffset': 0}, 'sub_obj_metadata': None}
{'obj_id': 32110640314422712626398175952661658, 'metadata': {'vdim': 0}, 'sub_obj_metadata': None}
Single Metadata retrieval of a single sub-object : TPS=1482.4607630549767, MetaPS: 1482.4586671775135


### Retrieving multiple attributes from single sub-objects

In [11]:
meta_keys=["name",
            "type",
            "shape",
            "vcount",
            "voffset",
            "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[1],
        meta_keys=meta_keys, # retreiving SINGLE!!!
        
    )
    
pprint(result)
print("\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(1000/(time.time()-start), 1000*len(meta_keys)/(time.time()-start)))

{'metadata': {'name': '689609/arr6',
              'shape': [1, 1, 1, 2, 5],
              'type': 'array',
              'vcount': 100,
              'vdim': 0,
              'voffset': 0},
 'obj_id': 32110640314422712626398175952661658,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of a single sub-object : TPS=1378.5779129304128, MetaPS: 8271.44572840571


### Retrieving multiple attributes from multiple sub-objects

In [12]:
meta_keys=["name",
            "type",
            "shape",
            "vcount",
            "voffset",
            "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys=meta_keys, # retreiving MULTIPLE for every sub-object!!!     
    )

pprint(result)
print("Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(1000/(time.time()-start), 1000*len(meta_keys)*3/(time.time()-start)))

{'metadata': {},
 'obj_id': 32110640314422103883843739242491034,
 'sub_obj_metadata': [{'metadata': {'name': '689609/arr5',
                                    'shape': [1, 1, 1, 2, 5],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '689609/arr5',
                       'obj_id': 32110640314422970880815212181351578},
                      {'metadata': {'name': '689609/gnn_arr1',
                                    'shape': [100000, 2],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '689609/gnn_arr1',
                       'obj_id': 32110640314433356397728723543813274},
                      {'metadata': {'name': '689609

### Retrieving different set of attributes from multiple sub-objects

In [13]:
# meta_keys=["name",
#             "type",
#             "shape",
#             "vcount",
#             "voffset",
#             "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys={"{}/arr5".format(name_id):[ "name", "type", "voffset" ],
                      "{}/arr4".format(name_id):["type", "vcount"]}      # retrieving different set of metadata for different sub-objects
    )

pprint(result)
print("\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(1000/(time.time()-start), 1000*5/(time.time()-start)))

{'metadata': {},
 'obj_id': 32110640314422103883843739242491034,
 'sub_obj_metadata': [{'metadata': {'type': 'array', 'vcount': 100},
                       'name': '689609/arr4',
                       'obj_id': 32110640314423173795000027281386650},
                      {'metadata': {'name': '689609/arr5',
                                    'type': 'array',
                                    'voffset': 0},
                       'name': '689609/arr5',
                       'obj_id': 32110640314422970880815212181351578}]}

 Multiple Metadata retrieval of a single sub-object : TPS=1301.219067651352, MetaPS: 6506.087264598003


### Retrieving array of a single object

In [19]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 1), slice(0, 2, -1)],
        
    )
pprint(result)
print("\n retrieving array slice of a single 3D array, TPS={}, Sample/s = {}".format(1000/(time.time()-start), (1000/6)/(time.time()-start)))

{'array_slice': array([[[0.04777806, 0.01492931]]]), 'sub_obj_slices': []}

 retrieving array slice of a single 3D array, TPS=1211.4926911327705, Sample/s = 201.91521523545438


### Retrieving array slices of multiple sub-objects (Large subarray included)

In [20]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 2), slice(0, 2, -1)],
        sub_obj_regions=[
            (
                "{}/arr5".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
            (
                "{}/arr4".format(name_id),
                [slice(0, 2, -1), slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2, -1)],
            ),
            (
                "{}/arr3".format(name_id),
                [slice(0, 1), slice(0, 2), slice(0, 10, -2)],
            ),
            (
                "{}/arr6".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
        ]
    )
pprint(result)
print("\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(1000/(time.time()-start), (1000/(4/6))/(time.time()-start)))

{'array_slice': array([[[0.04777806, 0.01492931],
        [0.080134  , 0.50924195]]]),
 'sub_obj_slices': [{'array': array([[[[[1.45058269, 0.85441015],
          [0.95909726, 1.06440206]]]]]),
                     'id': 32110640314422970880815212181351578,
                     'name': '689609/arr5'},
                    {'array': array([[[[[1.42722742, 0.28197623]]]],



       [[[[0.78869064, 1.13682084]]]]]),
                     'id': 32110640314423173795000027281386650,
                     'name': '689609/arr4'},
                    {'array': array([[[0.82158399, 0.4114978 , 0.47877315, 0.45233998, 0.04777806],
        [0.72869339, 0.62328352, 0.28240017, 0.88111086, 0.080134  ]]]),
                     'id': 32110640314433208823776129572433050,
                     'name': '689609/arr3'},
                    {'array': array([[[[[2.90116538, 1.7088203 ],
          [1.91819452, 2.12880412]]]]]),
                     'id': 32110640314422712626398175952661658,
                     '

### Retrieving slices of multiple objects (only GNN arrays)

In [23]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=None,
        sub_obj_regions=[
            (
                "{}/gnn_arr1".format(name_id),
                [slice(0, 20, -1), slice(0)],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [slice(0, 10, -1), slice(0)],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [slice(0, 15, -1), slice(0)],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [slice(0), slice(0, 20, -1)],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [slice(0), slice(0, 20, -1)],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [slice(0), slice(0, 20, -1)],
            ),
              
        ]
    )
pprint(result)
print("\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(1000/(time.time()-start), (1000)/(time.time()-start)))

{'array_slice': None,
 'sub_obj_slices': [{'array': array([], shape=(20, 0), dtype=float64),
                     'id': 32110640314433356397728723543813274,
                     'name': '689609/gnn_arr1'},
                    {'array': array([], shape=(10, 0), dtype=float64),
                     'id': 32110640314433356397728723543813274,
                     'name': '689609/gnn_arr1'},
                    {'array': array([], shape=(15, 0), dtype=float64),
                     'id': 32110640314433356397728723543813274,
                     'name': '689609/gnn_arr1'},
                    {'array': array([], shape=(0, 20), dtype=float64),
                     'id': 32110640314436086515851636852419738,
                     'name': '689609/gnn_arr2'},
                    {'array': array([], shape=(0, 20), dtype=float64),
                     'id': 32110640314436086515851636852419738,
                     'name': '689609/gnn_arr2'},
                    {'array': array([], shape=(0, 20), dty

## Closing the Store

In [17]:
# server_process.terminate()
# server_process.wait()
# print("Server process terminated.")
